In [1]:
import csv
import os


CWD = os.getcwd()
STOCK_FILE = CWD + "/stock.csv"
STOCK_HEADERS = "nome,quantità,prezzo vendita,prezzo acquisto"
BALANCE_FILE = CWD + "/balance.txt"
BALANCE_FILE_SKELETON = "profitto_lordo 0\n" + "margine_netto_sulle_vendite 0\n" + "spese 0"


balance = {}
stock = {}


################################################################################


def load_data_from_files() -> None:
    """Load the stock data and shop balance data from files (which are created if not existing).

    Returns:
        None
    """

    load_stock_from_file()
    load_balance_from_file()


def load_stock_from_file() -> None:
    """Load the stock data from `STOCK_FILE` (which is created if not existing) into this software.

    Returns:
        None
    """

    global stock
    stock = {}

    if not os.path.exists(STOCK_FILE) or os.path.getsize(STOCK_FILE) == 0:
        with open(STOCK_FILE, "w") as stock_file:
            stock_file.write(STOCK_HEADERS+"\n")

    with open(STOCK_FILE) as stock_file:
        csv_reader = csv.DictReader(stock_file)

        for product_details in csv_reader:
            stock[product_details["nome"]] = {
                "name":       product_details["nome"],
                "quantity":   int(product_details["quantità"]),
                "buy_price":  float(product_details["prezzo acquisto"]),
                "sell_price": float(product_details["prezzo vendita"])
            }


def translate_en_it(field: str) -> str:
    """Translate the given field from english to italian.
    
    Args:
        field: The field to be translated from english to italian. Has to be
               one of 'gross_profit', 'net_margin_on_sales', 'expenses'.

    Returns:
        str: The italian translation of the given field.
    """

    if field == "gross_profit":
        return "profitto_lordo"
    elif field == "net_margin_on_sales":
        return "margine_netto_sulle_vendite"
    elif field == "expenses":
        return "spese"


def translate_it_en(field: str) -> str:
    """Translate the given field from italian to english.
    
    Args:
        field: The field to be translated from italian to english. Has to be
               one of 'profitto_lordo', 'margine_netto_sulle_vendite', 'spese'.
               
    Returns:
        str: The english translation of the given field.
    """

    if field == "profitto_lordo":
        return "gross_profit"
    elif field == "margine_netto_sulle_vendite":
        return "net_margin_on_sales"
    elif field == "spese":
        return "expenses"


def load_balance_from_file() -> None:
    """Load the shop balance data from `BALANCE_FILE` (which is created if not existing).

    Returns:
        None
    """

    global balance
    balance = {}

    if not os.path.exists(BALANCE_FILE) or os.path.getsize(BALANCE_FILE) == 0:
        with open(BALANCE_FILE, "w") as balance_file:
            balance_file.write(BALANCE_FILE_SKELETON)

    with open(BALANCE_FILE) as balance_file:

        for line in balance_file.readlines():
            balance_entry, amount = line.split()
            amount = float(amount)

            if balance_entry == "profitto_lordo":
                balance["gross_profit"] = amount
            elif balance_entry == "margine_netto_sulle_vendite":
                balance["net_margin_on_sales"] = amount
            elif balance_entry == "spese":
                balance["expenses"] = amount


def get_valid_product_name() -> str:
    """Get a valid product name (not empty, not numeric) for a product.

    Returns:
        str: The valid product name.
    """

    product_name = None
    error_msg = "Nome non valido! Riprovare."

    while product_name is None:
        try:
            product_name = input("Nome del prodotto: ")
            if product_name == "" or product_name.replace(" ", "").isnumeric():
                product_name = None
                print(error_msg)
        except Exception:
            print(error_msg)

    return product_name


def get_valid_quantity() -> int:
    """Get a valid quantity (positive integer) from user input for a product.

    Returns:
        int: The valid quantity.
    """

    quantity = None
    error_msg = "Quantità non valida! Riprovare."

    while quantity is None or quantity <= 0:
        try:
            quantity = int(input("Quantità: "))
            if quantity <= 0:
                quantity = None
                print(error_msg)
        except Exception:
            print(error_msg)

    return quantity


def get_valid_price(prompt_msg: str) -> float:
    """Get a valid price (positive number) from user input for a product.

    Args:
        prompt_msg: The message to specify that the program is expecting to
                    receive a price from user input.

    Returns:
        float: The valid price.
    """

    price = None
    error_msg = "Prezzo non valido! Riprovare."

    while price is None or price <= 0:
        try:
            price = float(input(prompt_msg))
            if price <= 0:
                print(error_msg)
        except Exception:
            print(error_msg)

    return price


def show_purchase_details(purchase_list: list[list]) -> None:
    """After completing a purchase, show each product bought, its quantity and unit price, and the total amount spent. 

    Args:
        purchase_list: list of lists, each one containing details regarding the
                       purchase of one type of product. Each entry in purchase_list
                       has the following structure:
                       [product_name: str, quantity: int, unit_price: float,
                       total_price: float]

    Returns:
        None
    """

    print("VENDITA REGISTRATA")
    for purchase_entry in purchase_list:
        product_name, quantity, unit_price, tot_price = purchase_entry
        print(f"{quantity} X {product_name}: €{unit_price:.2f}")

    print(f"Totale: €{sum(x[-1] for x in purchase_list):.2f}")


def is_purchase_finished() -> bool:
    """Ask the user if they want to add another product to the purchase.

    Returns:
        bool: True if the user wants to add another item to the purchase,
              False otherwise.
    """
    
    purchase_finished = None

    while purchase_finished is None:
        sell_another = input("Aggiungere un altro prodotto? (si/no): ")
        if sell_another == "no":
            purchase_finished = True
        elif sell_another == "si":
            purchase_finished = False
        else:
            print("Scelta non valida, riprovare.")

    return purchase_finished


def update_balance_file(field_to_update: str, amount_to_add: float) -> None:
    """Update the given field of the shop balance by adding the given amount to it.

    Args:
        field_to_update: The english name of the field to update in the shop balance.
        amount_to_add: The amount of money to add to the given field.
    
    Returns:
        None
    """

    field_to_update = translate_en_it(field_to_update)
    lines = []

    with open(BALANCE_FILE) as balance_file:
        lines = balance_file.readlines()

        for i, line in enumerate(lines):

            if line.startswith(field_to_update):
                entry, amount = line.split()
                new_amount = str(float(amount) + amount_to_add)
                new_line = line.replace(amount, new_amount)
                # print(amount, new_amount, new_line)
                lines[i] = new_line

    with open(BALANCE_FILE, "w") as balance_file:
        balance_file.writelines(lines)


def add_to_stock_file(is_new_product: bool, product_name: str, quantity: int,
                      sell_price: float=None, buy_price: float=None) -> None:
    """Add one product to stock, and update the shop balance accordingly.

    Args:
        is_new_product: Indicates whether the product is already in stock.
        product_name: The name of the product to add to stock.
        quantity: The number of items of the product to add to stock.
        sell_price: The price (per unit) at which you'll sell the product.
        buy_price: The price (per unit) at which you bought the product.

    Returns:
        None
    """

    if is_new_product:
        with open(STOCK_FILE, "a") as stock_file:
            csv_writer = csv.writer(stock_file)
            csv_writer.writerow([product_name, quantity, sell_price, buy_price])
    else:
        TMP_FILE_FOR_WRITING = STOCK_FILE + ".tmp.csv"

        with open(STOCK_FILE) as stock_file_in, open(TMP_FILE_FOR_WRITING, "w") as stock_file_out:
            csv_reader = csv.reader(stock_file_in)
            csv_writer = csv.writer(stock_file_out)

            for line in csv_reader:
                if line[0] == product_name:
                    line[1] = int(line[1]) + quantity

                csv_writer.writerow(line)

            os.remove(STOCK_FILE)
            os.rename(TMP_FILE_FOR_WRITING, STOCK_FILE)


def remove_from_stock_file(product_name: str, quantity: int) -> None:
    """Remove the given quantity of a product from `STOCK_FILE`.

    Args:
        product_name: The name of the product to remove from `STOCK_FILE`.
        quantity: The number of items of the product to remove from `STOCK_FILE`.

    Returns:
        None
    """

    TMP_FILE_FOR_WRITING = STOCK_FILE + ".tmp.csv"

    with open(STOCK_FILE) as stock_file_in, open(TMP_FILE_FOR_WRITING, "w") as stock_file_out:
        csv_reader = csv.reader(stock_file_in)
        csv_writer = csv.writer(stock_file_out)

        headers = next(csv_reader)
        csv_writer.writerow(headers)

        for line in csv_reader:
            if line[0] == product_name:
                line[1] = int(line[1]) - quantity
            if int(line[1]) > 0:
                csv_writer.writerow(line)

        os.remove(STOCK_FILE)
        os.rename(TMP_FILE_FOR_WRITING, STOCK_FILE)


def is_product_in_stock(product_name: str) -> bool:
    """Check whether the given product is in stock.
    
    Args:
        product_name: The name of the product that needs to be checked.
    
    Returns:
        bool: True if the product is in stock, False otherwise.
    """

    return product_name in stock.keys()


def add_new_product_to_stock(product_name: str, quantity: int, sell_price: float,
                             buy_price: float) -> None:
    """Add to stock a product that was not in stock, and update the shop expenses.

    Args:
        product_name: The name of the product to add to stock.
        quantity: The number of items of the product to add to stock.
        sell_price: The price (per unit) at which you'll sell the product.
        buy_price: The price (per unit) at which you bought the product.

    Returns:
        None
    """

    stock[product_name] = {
        "name": product_name, "quantity": quantity, "sell_price": sell_price,
        "buy_price": buy_price
    }

    money_spent = quantity * buy_price
    balance["expenses"] += money_spent

    add_to_stock_file(is_new_product=True, product_name=product_name,
                      quantity=quantity, sell_price=sell_price, buy_price=buy_price)
    update_balance_file(field_to_update="expenses", amount_to_add=money_spent)


def add_existing_product_to_stock(product_name: str, quantity: int) -> None:
    """Add `quantity` items of `product_name` (which has been verified to be
    already in stock) to the stock, and update the shop expenses.

    Args:
        product_name: The name of the product.
        quantity: The item count of the product.

    Returns:
        None
    """

    stock[product_name]["quantity"] += quantity

    money_spent = quantity * stock[product_name]["buy_price"]
    balance["expenses"] += money_spent

    add_to_stock_file(is_new_product=False, product_name=product_name, quantity=quantity)
    update_balance_file(field_to_update="expenses", amount_to_add=money_spent)


def sell_product_from_stock(product_name: str, quantity: int) -> list:
    """Sell `quantity` of `product_name` from the stock, update the shop balance and update the stock.

    Args:
        product_name: The name of the product to sell.
        quantity: The number of items to sell.

    Returns:
        A list containing the details of the purchase. For example:
        ["Bistecca", 10, 4.90, 49]
    """

    if not is_product_in_stock(product_name):
        print("Prodotto non disponibile")
        return []

    if stock[product_name]["quantity"] < quantity:
        print("Errore: sono disponibili {} prodotti (richiesti {}). Vendita non effettuata.".format(
            stock[product_name]["quantity"], quantity
        ))
        return []

    # Update balance
    unit_price = stock[product_name]["sell_price"]
    cumulative_price = quantity * unit_price
    balance["gross_profit"] += cumulative_price

    cumulative_cost = quantity * stock[product_name]["buy_price"]
    balance["net_margin_on_sales"] += cumulative_price - cumulative_cost

    update_balance_file(field_to_update="gross_profit", amount_to_add=cumulative_price)
    update_balance_file(field_to_update="net_margin_on_sales", amount_to_add=cumulative_price-cumulative_cost)

    # Update stock
    if stock[product_name]["quantity"] == quantity:
        del stock[product_name]
    else:
        stock[product_name]["quantity"] -= quantity

    remove_from_stock_file(product_name, quantity)

    return [product_name, quantity, unit_price, cumulative_price]


###############################################################################
### All high-level functions are below
###############################################################################


def show_help_message() -> None:
    """Show a help message with all the available commands.
    
    Returns:
        None
    """

    print(
        """I comandi disponibili sono i seguenti:
        - aggiungi: aggiungi un prodotto al magazzino
        - elenca: elenca i prodotto in magazzino
        - vendita: registra una vendita effettuata
        - profitti: mostra i profitti totali
        - aiuto: mostra i possibili comandi
        - chiudi: esci dal programma
        """
    )


def menu_add_product_to_stock() -> None:
    """Get one product from user input, add it to the stock and update the shop expenses.
    
    Returns:
        None
    """

    product_name = get_valid_product_name()
    quantity = get_valid_quantity()

    if is_product_in_stock(product_name):
        add_existing_product_to_stock(product_name, quantity)
    else:
        buy_price = get_valid_price(prompt_msg="Prezzo d'acquisto: ")
        sell_price = get_valid_price(prompt_msg="Prezzo di vendita: ")
        add_new_product_to_stock(product_name, quantity, sell_price, buy_price)

    print(f"AGGIUNTO: {quantity} X {product_name}")


def list_all_available_products() -> None:
    """List all the available products in stock.
    
    Returns:
        None
    """

    if not stock:
        print("Nessun prodotto presente in magazzino.")
        return

    print("PRODOTTO QUANTITÀ PREZZO")

    for product_details in stock.values():
        print(product_details["name"],
              product_details["quantity"],
              "€%.2f" % product_details["sell_price"])


def menu_sell_products() -> None:
    """Get products to purchase from user input and update the stock and the shop balance accordingly.
    
    Returns:
        None
    """

    purchase_list = []
    purchase_finished = False

    while not purchase_finished:
        product_name = get_valid_product_name()
        quantity = get_valid_quantity()

        purchase = sell_product_from_stock(product_name, quantity)
        if purchase:
            purchase_list.append(purchase)

        purchase_finished = is_purchase_finished()

    if purchase_list:
        show_purchase_details(purchase_list)
    else:
        print("Nessun prodotto comprato.")


def list_all_profits() -> None:
    """List gross and net profits for the shop.
    
    Returns:
        None
    """

    gross_profit = float(balance["gross_profit"])
    net_margin_on_sales = float(balance["net_margin_on_sales"])

    print("Profitto: lordo = €{:.2f}, netto = €{:.2f}".format(
        gross_profit, net_margin_on_sales
    ))


def close_product_management_software() -> None:
    """Exit the software after showing a message.
    
    Returns:
        None
    """

    print("Bye bye")
    exit


################################################################################
################################################################################
################################################################################
### di seguito il "main"


load_data_from_files()
cmd = None

show_help_message()

while cmd != "chiudi":
    cmd = input("Inserisci un comando: ")

    if cmd == "aiuto":
        show_help_message()

    elif cmd == "aggiungi":
        menu_add_product_to_stock()

    elif cmd == "elenca":
        list_all_available_products()

    elif cmd == "vendita":
        menu_sell_products()

    elif cmd == "profitti":
        list_all_profits()

    elif cmd == "chiudi":
        close_product_management_software()

    else:
        print("\nComando non valido.")
        show_help_message()


Comando non valido.
I comandi disponibili sono i seguenti:
        - aggiungi: aggiungi un prodotto al magazzino
        - elenca: elenca i prodotto in magazzino
        - vendita: registra una vendita effettuata
        - profitti: mostra i profitti totali
        - aiuto: mostra i possibili comandi
        - chiudi: esci dal programma
        

Comando non valido.
I comandi disponibili sono i seguenti:
        - aggiungi: aggiungi un prodotto al magazzino
        - elenca: elenca i prodotto in magazzino
        - vendita: registra una vendita effettuata
        - profitti: mostra i profitti totali
        - aiuto: mostra i possibili comandi
        - chiudi: esci dal programma
        
